# Monstrinho 3: Classes em Python não pagam imposto sobre herança

**Objetivo:** Modele algum conceito científico utilizando herança de classes.

**Considerações do experimento:**  O uso da herança de classes deve fazer sentido dentro do contexto científico escolhido, isto é, deve haver uma justificativa bem embasada para o uso de herança de classes na sua entrega. Certifique-se que a classe mãe tem pelo menos um método que não seja dunder para ser herdado pela classe filha. Garanta que a classe filha tem pelo menos um método (dunder ou não) que justifique a sua criação.

**Resolução:**

O uso de classes para a verificação de taxonomia de várias espécies é muito interessante. Dentre os organismos que podem ser modelados dessa maneira, os vírus são interessantes por apresentar características genômicas que tem relevância de cálculo, como a taxa de mutação por infecção em hospedeiros e suas capacidades de reparação gênica.

Sabe-se que há uma ampla gama de tipos virais [1], mas, para os objetivos dessa tarefa, decidiu-se focar em caracterizar os vírus entre "ssRNA" (vírus de RNA de fita simples), "dsRNA" (vírus de RNA de fita dupla), "ssDNA" (vírus de DNA de fita simples) e "dsDNA" (vírus de DNA de fita dupla). A superclasse foi definida como ``Virus``, e tem como métodos:
- ``__init__(self, sequence: str, genetic_material: str, name = "Desconhecido", host = "Desconhecido")``: inicia a instância com os atributos da sequência de nucleotídeos, o tipo de material genético e, opcionalmente, o nome e o hospedeiro do vírus. Além disso, nessa classe, é instanciado o atributo de tamanho do genoma (``self.size``), o número de mutações (``self.number_mutations``) e a última sequência antes de alguma mutação (``self.old_sequence``).
- ``__repr__``: define o que vai ser printado.
- ``__check_genetics__(self, genetic_material)``: método ``__dunder__`` que checa se o material genético informado está entre os aceitos. É chamado na inicialização da classe, e joga uma exceção quando o material informado não se adequa.
- ``__format_seq__(self, sequence)``: formata a sequência informada para obedecer ao padrão pré-definido de letras maiúsculas e a presença do nucleotídeo "U" (uracila) caso o vírus seja de RNA.
- ``infection(self, weight=(1-0.0001, 0.0001))``: método que representa uma infecção do vírus em seu hospedeiro. O peso está relacionado com a probabilidade de ocorrer mutação na infecção, e seu valor padrão é a média encontrada no artigo [2] para vírus de DNA e RNA.
- ``mutate``: método que representa uma mutação ocorrida no genoma do vírus. Os pesos atribuídos são referentes à probabilidade de ocorrer uma translocação/inversão (aqui admitidas como tendo mesma probabilidade, apesar de não necessariamente representar a realidade) ou um inserção/deleção. A probabilidade foi retirada de [2].

In [10]:
from random import choices # importação da biblioteca choices

In [11]:
class Virus:
    """
    Instancia um objeto na classe Vírus, com o método principal "mutate()", que modifica seu material genético.
    """

    def __init__(self, sequence: str, genetic_material: str, name = "Desconhecido", host = "Desconhecido"):
        
        # Checa se o material genético está entre os aceitos pela classe. Se não estiver, joga uma exceção.
        self.__check_genetics__(genetic_material.replace(" ", "").lower())

        # Definição dos atributos mais simples do vírus
        self.name = name # Nome do vírus
        self.genetic_material = genetic_material.replace(" ", "").lower() # ssDNA, dsDNA, ssRNA, dsRNA
        self.host = host # Hospedeiro do vírus
        self.sequence = self.__format_seq__(sequence) # Sequência de nucleotídeos
        self.size = len(self.sequence) # Tamanho da sequência viral

        # Atributos que fazem sentido com o método mutate() abaixo.
        self.number_mutations = 0 # Número de mutações que o objeto já passou.
        self.old_sequence = self.sequence # Sequência no momento da instância
    
    def __repr__(self):
        genetics = {
            "dsdna" : "DNA de fita dupla",
            "ssdna": "DNA de fita simples",
            "dsrna": "RNA de fita dupla",
            "ssrna" : "RNA de fita simples"
        }

        return f"Vírus {self.name} com material genético {genetics[self.genetic_material]} e {self.size} bases pareadas."
    
    def __check_genetics__(self, genetic_material):
        if genetic_material in ["ssdna", "dsdna", "ssrna", "dsrna"]:
            return
        else: raise ValueError("The accepted values are 'ssDNA', 'dsDNA', 'ssRNA', 'dsRNA'.")
        
    def __format_seq__(self, sequence):
        seq = sequence.replace(" ", "").replace("\n", "").replace("\t", "").upper()

        if self.genetic_material.find("DNA") != -1:
            seq = seq.replace("U", "T")
        else:
            seq = seq.replace("T", "U")

        return seq

    def infection(self, weight=(1-0.0001, 0.0001)):
        mutation = ""
        mutation_chance = choices([0,1], weight)

        if mutation_chance == 1:
            self.mutate()
            mutation += " Houve uma mutação."
        else:
            pass

        return f"O vírus infectou o hospedeiro {self.host}.{mutation}"


    def mutate(self):
        self.old_sequence = self.sequence # Sequência antes da mutação
        self.number_mutations += 1

        index_mutation = choices([i for i in range(0, self.size-1)])
        mut_nucleotide = self.sequence[index_mutation]

        if self.genetic_material.find("dna") != -1:
            possible_nucleotides = ["A", "T", "G", "C"]
            possible_nucleotides.remove(mut_nucleotide)
            possible_nucleotides = possible_nucleotides + [""]

            mut = choices(possible_nucleotides, weights=(8/30, 8/30, 8/30, 6/30))

        elif self.genetic_material.find("rna") != -1:
            possible_nucleotides = ["A", "U", "G", "C"]
            possible_nucleotides.remove(mut_nucleotide)
            possible_nucleotides = possible_nucleotides + [""]

            mut = choices(possible_nucleotides, weights=(8/30, 8/30, 8/30, 6/30))

        self.sequence = self.sequence[:index_mutation] 
        + mut 
        + self.sequence[choices([index_mutation+1, index_mutation], weights=(24/30, 6/30)):]

        self.size = len(self.sequence)
        
        return self.sequence, index_mutation

A subclasse foi definida como ``DNAVirus``, e se refere especificamente a esse tipo de vírus. Todos os atributos da superclasse se aplicam, além do novo ``number_repairs``, que vai indicar a quantidade de reparos que o vírus passou. Além dos métodos herdados, modificou-se os métodos ``__check_genetics__`` e ``infection``, além de definir o método ``genetic_repair``.
- ``__check_genetics__(self, genetic_material)``: modificou-se para aceitar apenas "ssDNA" e "dsDNA" como materiais genéticos.
- ``infection``: a mudança ocorrida nesse método foi a adição da possibilidade de ocorrer um reparo no genoma do vírus e a diminuição da probabilidade de ocorrer uma mutação no genoma, obtida em [2].
- ``genetic_repair``: método que representa o retorno à sequência anterior à última mutação, representando o mecanismo de reparo presente em vírus de DNA.

In [6]:
class DNAVirus(Virus):
    def __init__(self, sequence: str, genetic_material: str, name = "Desconhecido", host = "Desconhecido") -> None:

        # Checa se o material genético está entre os aceitos pela classe. Se não estiver, joga uma exceção.
        self.__check_genetics__(genetic_material.replace(" ", "").lower())

        # Inicia a classe Virus
        super().__init__(sequence, genetic_material, name, host)

        self.number_repairs = 0 # Números de reparos genéticos.
    
    def __check_genetics__(self, genetic_material):
        if genetic_material in ["ssdna", "dsdna"]:
            return
        else: raise ValueError("The accepted values are 'ssDNA', 'dsDNA'.")
    
    def infection(self):
        infect = super().infection(weight=(1-0.0000001, 0.0000001))
        repair = ""

        if infect.find("Houve uma mutação.") != -1:

            repair_chance = choices([0,1])
            if repair_chance == 1:
                self.genetic_repair()
                repair += " Houve um reparo."

            else:
                pass

        return f"{infect}{repair}"

    def genetic_repair(self):

        """
        Repara a última mutação.
        """
        self.number_repairs += 1
        self.sequence = self.old_sequence

        return self.sequence

Nessa atividade, foi possível compreender como trabalhar o conceito de herança. Gostei de conseguir aplicar a um conceito cientifico de meu interesse, e possivelmente poderei ampliar esse código. Para além da disciplina de RNAG, pude ainda aprender um pouco mais sobre replicação e mutação de vírus, o que é bem interessante. Foi ótimo!

**Referências:**

[1] GELDERBLOM, Hans R. Structure and Classification of Viruses. In: BARON, Samuel (Org.). Medical Microbiology. 4th. ed. Galveston (TX): University of Texas Medical Branch at Galveston, 1996. Disponível em: <http://www.ncbi.nlm.nih.gov/books/NBK8174/>. Acesso em: 15 mar. 2025.

[2] SANJUÁN, Rafael; NEBOT, Miguel R.; CHIRICO, Nicola; et al. Viral Mutation Rates. Journal of Virology, v. 84, n. 19, p. 9733–9748, 2010. Disponível em: <https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2937809/>. Acesso em: 15 mar. 2025.